<a href="https://colab.research.google.com/github/mudu89/ML_AssignmentPS01/blob/main/2024DA04200_Problem_Statement_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/ML_Assignment_01/ML_AssignmentPS01

/content/drive/MyDrive/ML_Assignment_01/ML_AssignmentPS01


1. Importing the dataset:

Read the dataset from the path into a pandas dataframe.

In [4]:
import pandas as pd
dataset_file = './risk_factors_cervical_cancer.csv'

df = pd.read_csv(dataset_file)

2. Data Exploration:

In this section, we will perform exploratory analysis of the dataset to get insight of the data, and understand the correlation of the variables.

We will also identify if the correlation analysis helps in the feature selection process.

In [5]:
pd.set_option('display.max_columns', None)
df.head(2)

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,IUD (years),STDs,STDs (number),STDs:condylomatosis,STDs:cervical condylomatosis,STDs:vaginal condylomatosis,STDs:vulvo-perineal condylomatosis,STDs:syphilis,STDs:pelvic inflammatory disease,STDs:genital herpes,STDs:molluscum contagiosum,STDs:AIDS,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0


In [6]:
df.dtypes

,0
Age,int64
Number of sexual partners,object
First sexual intercourse,object
Num of pregnancies,object
Smokes,object
Smokes (years),object
Smokes (packs/year),object
Hormonal Contraceptives,object
Hormonal Contraceptives (years),object
IUD,object


In [8]:
plt.figure(figsize=(5,4))
sns.countplot(x='Biopsy', data=numeric_df)
plt.title("Biopsy Distribution")
plt.show()